# support function

In [2]:
%run "../2016-12-16/RoofGenerator.ipynb"

In [131]:
# PIANTE
fronde01 = "./texture/albero01/pianta.jpg"
tr01 = "./texture/albero01/tronco.jpg"

fronde02 = "./texture/albero02/pianta.jpg"
tr02 = "./texture/albero02/tronco.jpg"

fronde03 = "./texture/albero03/pianta.jpg"
tr03 = "./texture/albero03/tronco.jpg"
# STRADA
asfalto = "./texture/strada/asfalto.jpg"

In [150]:
"""create normal tree"""
def createTree01(scale):
    scale = 0.08*scale
    fronda = DODECAHEDRON
    fronda = R([1,3])(PI/8)(fronda)
    fronda = centra(fronda)
    fronda = T(3)(1.8)(fronda)
    tronco = CYLINDER ([0.2,2.0])(8)
    fronda = DIFF([fronda, tronco])
    fronda = TEXTURE([fronde01, TRUE, TRUE,10,10,0,1,1,0,0])(fronda)
    tronco = TEXTURE([tr01, TRUE, FALSE,0,0,0,1,1,0,0])(tronco)
    tronco = centra(tronco)
    albero = STRUCT([fronda, tronco])
    albero = S([1,2,3])([scale,scale,scale])(albero)
    return albero

In [148]:
"""create cypress tree"""
def createTree02(scale):
    scale = 0.08*scale
    fronda = CONE([0.6, 4.0])(8)
    fronda = centra(fronda)
    fronda = T(3)(1.8)(fronda)
    tronco = CYLINDER ([0.2,2.0])(8)
    fronda = DIFF([fronda, tronco])
    fronda = TEXTURE([fronde02, TRUE, TRUE,10,10,0,1,1,0,0])(fronda)
    tronco = TEXTURE([tr02, TRUE, FALSE,0,0,0,1,1,0,0])(tronco)
    tronco = centra(tronco)
    albero =  STRUCT([fronda, tronco])
    albero = S([1,2,3])([scale,scale,scale])(albero)
    return albero

In [149]:
"""create pine tree"""
def createTree03(scale):
    scale = 0.08*scale
    fronda = DODECAHEDRON
    fronda = R([1,3])(PI/3)(fronda)
    fronda = centra(fronda)
    fronda = T(3)(1.8)(fronda)
    fronda = S([1,2])([2,2])(fronda)
    tronco = CYLINDER ([0.2,2.0])(8)
    fronda = DIFF([fronda, tronco])
    fronda = TEXTURE([fronde03, TRUE, TRUE,10,10,0,1,1,0,0])(fronda)
    tronco = TEXTURE([tr03, TRUE, FALSE,0,0,0,1,1,0,0])(tronco)
    tronco = centra(tronco)
    albero =  STRUCT([fronda, tronco])
    albero = S([1,2,3])([scale,scale,scale])(albero)
    return albero

In [146]:
"""create tree from .lines"""
def readForest(file_name):
    i = 0
    forest = []
    with open(file_name, 'rb') as csvfile:
        buildReader = csv.reader(csvfile)
        for row in buildReader:
            x1 = float(row[0])
            y1 = float(row[1])
            x2 = float(row[2])
            y2 = float(row[3])
            i += 2
            
            t = random.randint(0, 3)
            s = (random.randrange(80, 120, 5)/10.0)
            a = angle(x1, y1, x2, y2)
            
            if(t == 0):
                tree = createTree01(s)
            else:
                if(t == 1):
                    tree = createTree02(s)
                else:
                    tree = createTree03(s)
                    
            tree = R([1,2])(a)(tree)
            tree = T([1,2])([x1, y1])(tree)
            forest.append(tree)
    
        forest = STRUCT(forest)
    #forest = R([1,2])(PI)(forest)
    #size = sizeOf(forest)
    #forest = T([1,2])([size[0], size[1]])(forest)
    return forest

In [10]:
"""create houses from .lines"""
def readHouses(file_name):
    i = 0
    struct = []
    with open(file_name, 'rb') as csvfile:
        buildReader = csv.reader(csvfile)
        
        for row in buildReader:
            x1 = float(row[0])
            y1 = float(row[1])
            x2 = float(row[2])
            y2 = float(row[3])
            i += 2
            a = angle(x1, y1, x2, y2)
            randomFloor = random.randint(1, 11)
            casa = createPalace(randomFloor)
            size = sizeOf(casa)
            casa = R([1,2])(a)(casa)
            casa = T([1,2])([x1, y1])(casa)
            struct.append(casa)
    return STRUCT(struct)

In [6]:
"""Special function, read a specific illustrator format AI3 and generate bezier curve"""
def readIllustratorFile(file_name):
    points = []
    group = []
    reading = False
    with open(file_name, 'rU') as csvfile:
        buildReader = csv.reader(csvfile, delimiter=' ')
        for row in buildReader:
            if(row[0]=='1' and row[1]=='XR'):
                reading = True
                continue
            if(row[0]=='N' or row[0]=='n'):
                if len(points)>0:
                    group.append(points)
                points = []
                reading = False
                continue
            if reading:
                points.append([float(row[0]), float(row[1])])
                if isfloat(row[2]):
                    points.append([float(row[2]), float(row[3])])
                if len(row)>4 and isfloat(row[4]):
                    points.append([float(row[4]), float(row[5])])
    return group

In [7]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [133]:
"""create roads from .lines"""
def createRoads(path, width):
    points = readIllustratorFile(path)
    domain=INTERVALS(1)(100)
    roads = []
    for i in range(len(points)):
        appoggio = BEZIERSTRIPE([points[i],width,22])
        roads.append(appoggio)
    roads = STRUCT(roads)
    return roads


In [25]:
"""create river from .lines"""
def createRiver(path, width):
    river = createRoads(path, width)
    return river

In [11]:
"""tools"""
def creaCubi(file_name):
    i = 0
    struct = []
    with open(file_name, 'rb') as csvfile:
        buildReader = csv.reader(csvfile)
        for row in buildReader:
            x1 = float(row[0])
            y1 = float(row[1])
            x2 = float(row[2])
            y2 = float(row[3])
            i += 2
            a = angle(x1, y1, x2, y2)
            cubo = R([1,2])(a)(CUBOID([3,3,3]))
            cubo01 = COLOR(RED)(T([1,2])([x1, y1])(cubo))
            cubo02 = COLOR(BLUE)(T([1,2])([x2, y2])(cubo))
            struct.append(cubo01)
            struct.append(cubo02)
    return STRUCT(struct)

3